# <center> TensorFlow and Deep Learning </center>

## Text and Sequence Data

<br>

<center> ![](timeseq1.png) </center>

*<p style="text-align:right">By Glen Fergus [CC BY 3.0] via Wikimedia Commons</p>*



- build recurrent neural networks **(RNN)** for **NLP** (Natural Language Processing) and **NLU** (Natural Language Understanding)
- *NLP and NLU tasks involve: document classification, sentiment analysis, conversational agents*
- Tensorboard for visualization purposes
- ? Long Short-Term Memory models

### Sequence Data

- video - images arranged over time
- medical records
- temperature data (see above)
- audio data 
- stock market data
- etc.

## <center> Recurrent Neural Networks </center> 

<br>

<center> ![](rnnl.png) </center>

*<p style="text-align:right">By François Deloche [CC BY-SA 4.0] via Wikimedia Commons</p>*

- each node adds new information => updates the model
- in humans: learning new things => updates our model of the world
- **Markov Chain** model: data = sequence of (dependent) chains
- at time 't': xt, ht, ot
- the RNN: unrolled chain of nodes
- **supervised learning:** input node => activation (h) - supervisor-given target activations => some outputs
- **reinforcement learning:** fitness/reward function => influences inputs via outputs connected to actuators

## Implementing MNIST for RNNs

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

element_size = 28
time_steps = 28
num_classes = 10 
batch_size = 128
hidden_layer_size = 128 

LOG_DIR = 'logs/RNN_with_summaries'

_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,element_size], name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')

batch_x, batch_y = mnist.train.next_batch(batch_size)
batch_x = batch_x.reshape((batch_size, time_steps,element_size))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
def var_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
with tf.name_scope('mn_weights'):
    with tf.name_scope('W_x'):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        var_summaries(Wx)
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        var_summaries(Wh)
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        var_summaries(b_rnn)

In [3]:
def rnn_step(previous_hidden_state,x):
    current_hidden_state = tf.tanh(tf.matmul(previous_hidden_state, Wh)+tf.matmul(x,Wx)+b_rnn)
    return current_hidden_state

In [4]:
processed_input = tf.transpose(_inputs, perm=[1,0,2])
initial_hidden = tf.zeros([batch_size, hidden_layer_size])
all_hidden_states = tf.scan(rnn_step, processed_input, initializer=initial_hidden, name='states')

with tf.name_scope('linear_layer_weights') as scope:
        with tf.name_scope('W_linear'):
            Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=.01))
            var_summaries(Wl)
            bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))
            var_summaries(bl)
            
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

In [5]:
with tf.name_scope('linear_layer_weights') as scope:
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    output = all_outputs[-1]
    tf.summary.histogram('outputs',output)

In [6]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)
    
with tf.name_scope('train'):
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)
    
merged = tf.summary.merge_all()

In [7]:
test_data = mnist.test.images[:batch_size].reshape((-1,time_steps,element_size))
test_label = mnist.test.labels[:batch_size]

with tf.Session() as sess:
    
    train_writer = tf.summary.FileWriter(LOG_DIR+'/train',graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR+'/test',graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size,time_steps,element_size))
        summary,_ = sess.run([merged,train_step],feed_dict={_inputs:batch_x, y:batch_y})
        train_writer.add_summary(summary,i)
        
        if i%1000 == 0:
            
            acc, loss, = sess.run([accuracy,cross_entropy], feed_dict={_inputs:batch_x, y:batch_y})
            print('Iter '+str(i)+', Minibatch Loss='+'{:.6f}'.format(loss)+', Training Accuracy='+'{:.5f}'.format(acc))
            
        if i%10:
            summary,acc = sess.run([merged,accuracy], feed_dict={_inputs:test_data,y:test_label})
            test_writer.add_summary(summary,i)
            
    test_acc = sess.run(accuracy,feed_dict={_inputs:test_data,y:test_label})
    print('Test Accuracy: ',test_acc)   
        
        
        

Iter 0, Minibatch Loss=2.301976, Training Accuracy=8.59375
Iter 1000, Minibatch Loss=1.205346, Training Accuracy=56.25000
Iter 2000, Minibatch Loss=0.611657, Training Accuracy=82.81250
Iter 3000, Minibatch Loss=0.264072, Training Accuracy=90.62500
Iter 4000, Minibatch Loss=0.261685, Training Accuracy=92.96875
Iter 5000, Minibatch Loss=0.074211, Training Accuracy=98.43750
Iter 6000, Minibatch Loss=0.156546, Training Accuracy=96.87500
Iter 7000, Minibatch Loss=0.116631, Training Accuracy=96.87500
Iter 8000, Minibatch Loss=0.046953, Training Accuracy=99.21875
Iter 9000, Minibatch Loss=0.141044, Training Accuracy=96.87500
Test Accuracy:  96.0938
